In [1]:
import numpy as np
import os
import glog as log
import copy

import open3d as o3d
from open3d import JVisualizer
import pandas as pd

from evaluation.tools.mesh import Mesh
from evaluation.tools.mesh_evaluator import MeshEvaluator

# Rotation matrices:
# East North Up (ENU) frame to Unity's world frame of reference
enu_R_unity = np.array([[1, 0, 0],
                        [0, 0, 1],
                        [0, 1, 0]])
unity_R_enu = np.transpose(enu_R_unity)

# Right Handed frame to Unity's Left Handed frame of reference
righthand_R_lefthand = np.array([[1, 0, 0],
                                 [0, -1, 0],
                                 [0, 0, 1]])
lefthand_R_righthand = np.transpose(righthand_R_lefthand)

gt_mesh_path = "/home/tonirv/Downloads/tesse_multiscene_office1_3d_semantic_v5.ply"
est_mesh_path = "/home/tonirv/Downloads/tesse_semantics_2.ply"

In [2]:
print("Loading Ground-truth mesh...")
gt_mesh_original = Mesh(gt_mesh_path)
print("Loading Estimated mesh...")
est_mesh_original = Mesh(est_mesh_path)

Loading Ground-truth mesh...
Testing mesh in open3d ...
geometry::TriangleMesh with 1572372 points and 524124 triangles.
[[14.5       4.       27.5     ]
 [13.5       4.       39.5     ]
 [14.5       4.       39.5     ]
 ...
 [ 6.567781  2.118583 36.00596 ]
 [ 6.568742  2.126512 36.21212 ]
 [ 6.568742  2.126512 36.00596 ]]
[[      0       1       2]
 [      3       4       5]
 [      6       7       8]
 ...
 [1572363 1572364 1572365]
 [1572366 1572367 1572368]
 [1572369 1572370 1572371]]

Loading Estimated mesh...
Testing mesh in open3d ...
geometry::TriangleMesh with 1275452 points and 1401701 triangles.
[[ 1.125       0.99864799 11.82499981]
 [ 1.17499995  0.992746   11.875     ]
 [ 1.17499995  0.996822   11.82499981]
 ...
 [-4.7750001   1.01013994 25.47500038]
 [-4.81760979  0.97500002 25.57500076]
 [-4.7750001   1.00861001 25.57500076]]
[[      0       1       2]
 [      3       1       0]
 [      4       5       6]
 ...
 [1275448 1275447 1275449]
 [1275450 1275451 1275446]
 [12754

In [3]:
# Transform Meshes to same frame of reference
gt_mesh = copy.deepcopy(gt_mesh_original)
est_mesh = copy.deepcopy(est_mesh_original)
est_mesh.mesh_o3d.translate([0, -5, 0])
gt_mesh.transform_left(righthand_R_lefthand)

Transforming mesh according to left matrix:
[[ 1  0  0]
 [ 0 -1  0]
 [ 0  0  1]]


In [4]:
vis = o3d.visualization.Visualizer()
vis.create_window()
vis.get_render_option().mesh_show_back_face = True
vis.add_geometry(est_mesh.mesh_o3d)
vis.add_geometry(gt_mesh.mesh_o3d)
vis.run()
vis.destroy_window()

In [5]:
gt_pcl = o3d.geometry.sample_points_uniformly(gt_mesh.mesh_o3d, 1000000)
est_pcl = o3d.geometry.sample_points_uniformly(est_mesh.mesh_o3d, 1000000)

In [6]:
# Calculate normals for nice visualization
o3d.geometry.estimate_normals(
        est_pcl,
        search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1,
                                                          max_nn=5))
o3d.geometry.estimate_normals(
        gt_pcl,
        search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1,
                                                          max_nn=5))

True

In [7]:
vis = o3d.visualization.Visualizer()
vis.create_window()
vis.get_render_option().mesh_show_back_face = True
vis.add_geometry(gt_pcl)
vis.add_geometry(est_pcl)
vis.run()
vis.destroy_window()

In [8]:
# ICP
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])
def draw_correspondences(source, target, correspondences):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    o3d.visualization.draw_geometries([source_temp, target_temp, correspondences])
    

In [9]:
# ICP params
ICP_THRESHOLD = 0.02
trans_init = np.asarray([[1.0, 0.0, 0.0, 0.0],
                         [0.0, 1.0, 0.0, 0.0],
                         [0.0, 0.0, 1.0, 0.0],
                         [0.0, 0.0, 0.0, 1.0]])

In [10]:
# Visualize initial registration problem
draw_registration_result(est_pcl, gt_pcl, trans_init)

In [14]:
# Evaluate current fit between pointclouds
evaluation = o3d.registration.evaluate_registration(est_pcl, gt_pcl, ICP_THRESHOLD, trans_init)

In [15]:
print("Initial registration")
print(evaluation)

Initial registration
registration::RegistrationResult with fitness = 0.091323, inlier_rmse = 0.014884, and correspondence_set size of 91323
Access transformation to get result.


In [16]:
print("Apply point-to-point ICP")
reg_p2p = o3d.registration.registration_icp(
    est_pcl, gt_pcl, ICP_THRESHOLD, trans_init,
    o3d.registration.TransformationEstimationPointToPlane())
correspondences = reg_p2p.correspondence_set

Apply point-to-point ICP


In [17]:
print(reg_p2p)
print("")

print("Transformation is:")
print(reg_p2p.transformation)
print("")

print("Correspondence Set:")
print(reg_p2p.correspondence_set)
print("")

registration::RegistrationResult with fitness = 0.109805, inlier_rmse = 0.014657, and correspondence_set size of 109805
Access transformation to get result.

Transformation is:
[[ 9.99998349e-01  4.38061490e-05  1.81676775e-03 -5.13051370e-02]
 [-4.39822951e-05  9.99999994e-01  9.69159334e-05  4.07717272e-03]
 [-1.81676349e-03 -9.69956790e-05  9.99998345e-01  2.93946067e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]

Correspondence Set:
std::vector<Eigen::Vector2i> with 109805 elements.
Use numpy.asarray() to access data.



In [18]:
# Draw Registration Result
draw_registration_result(est_pcl, gt_pcl, reg_p2p.transformation)

In [19]:
# Draw Only Correspondences
c2c_lines = o3d.geometry.create_line_set_from_point_cloud_correspondences(est_pcl, gt_pcl, correspondences)
o3d.visualization.draw_geometries([c2c_lines])

In [20]:
# Draw PointClouds and Correspondences
draw_correspondences(est_pcl, gt_pcl, c2c_lines)

In [94]:
total_negative_matches = 0
total_positive_matches = 0
total_correspondences = len(correspondences)
for correspondence in correspondences:
    est_pcl_color = est_pcl.colors[correspondence[0]]
    if np.array_equal(est_pcl_color, gt_pcl.colors[correspondence[1]]):
        total_positive_matches += 1
    else:
        total_negative_matches += 1
        
print "Negative color matches: ", total_negative_matches
print "Positive color matches: ",total_positive_matches
print "Total correspondences: ", total_correspondences
assert(total_correspondences == total_negative_matches + total_positive_matches)
print ("Negative {{ % }}: %5d"% total_negative_matches / total_correspondences * 100)
print ("Positive \%: %5d"% total_positive_matches / total_correspondences * 100)

Negative color matches:  85909
Positive color matches:  23896
Total correspondences:  109805


ValueError: unsupported format character '}' (0x7d) at index 14

In [21]:
# Import Semantic Labels
df = pd.read_csv('../metric_semantic_results/semantic_labels/tesse_multiscene_office1_segmentation_mapping.csv')
df

,Label,red,green,blue,alpha
0,Airventilation_3x,234,65,132,255
1,Airventilation_CrossSection,234,65,132,255
2,AirventilationGrid_A,234,65,132,255
3,AirventilationGrid_B,234,65,132,255
4,Books,63,69,138,255
5,Bookshelf,83,102,202,255
6,Bottle,216,236,152,255
7,Box_Black,85,243,186,255
8,Box_Cardboard,85,243,186,255
9,Box_Gray,85,243,186,255


In [22]:
normalized_df = copy.deepcopy(df)
normalized_df['normalized_red'] = df['red'] / 255
normalized_df['normalized_green'] = df['green'] / 255
normalized_df['normalized_blue'] = df['blue'] / 255
normalized_df

,Label,red,green,blue,alpha,normalized_red,normalized_green,normalized_blue
0,Airventilation_3x,234,65,132,255,0.917647,0.254902,0.517647
1,Airventilation_CrossSection,234,65,132,255,0.917647,0.254902,0.517647
2,AirventilationGrid_A,234,65,132,255,0.917647,0.254902,0.517647
3,AirventilationGrid_B,234,65,132,255,0.917647,0.254902,0.517647
4,Books,63,69,138,255,0.247059,0.270588,0.541176
5,Bookshelf,83,102,202,255,0.325490,0.400000,0.792157
6,Bottle,216,236,152,255,0.847059,0.925490,0.596078
7,Box_Black,85,243,186,255,0.333333,0.952941,0.729412
8,Box_Cardboard,85,243,186,255,0.333333,0.952941,0.729412
9,Box_Gray,85,243,186,255,0.333333,0.952941,0.729412


In [ ]:
from hashlib import sha1

from numpy import all, array, uint8


class hashable(object):
    r'''Hashable wrapper for ndarray objects.
        Instances of ndarray are not hashable, meaning they cannot be added to
        sets, nor used as keys in dictionaries. This is by design - ndarray
        objects are mutable, and therefore cannot reliably implement the
        __hash__() method.
        The hashable class allows a way around this limitation. It implements
        the required methods for hashable objects in terms of an encapsulated
        ndarray object. This can be either a copied instance (which is safer)
        or the original object (which requires the user to be careful enough
        not to modify it).
    '''
    def __init__(self, wrapped, tight=False):
        r'''Creates a new hashable object encapsulating an ndarray.
            wrapped
                The wrapped ndarray.
            tight
                Optional. If True, a copy of the input ndaray is created.
                Defaults to False.
        '''
        self.__tight = tight
        self.__wrapped = array(wrapped) if tight else wrapped
        self.__hash = int(sha1(wrapped.view(uint8)).hexdigest(), 16)

    def __eq__(self, other):
        return all(self.__wrapped == other.__wrapped)

    def __hash__(self):
        return self.__hash

    def unwrap(self):
        r'''Returns the encapsulated ndarray.
            If the wrapper is "tight", a copy of the encapsulated ndarray is
            returned. Otherwise, the encapsulated ndarray itself is returned.
        '''
        if self.__tight:
            return array(self.__wrapped)



In [78]:
red_col = df['red'].to_numpy()
blue_col = df['blue'].to_numpy()
green_col = df['green'].to_numpy()

def f(x):    
    return str(x['normalized_red'])+str(x['normalized_green'])+str(x['normalized_blue'])

hashed_df = copy.deepcopy(normalized_df)
hashed_df['hash'] = hashed_df.apply(f, axis=1)
hashed_df

,Label,red,green,blue,alpha,normalized_red,normalized_green,normalized_blue,hash
0,Airventilation_3x,234,65,132,255,0.917647,0.254902,0.517647,0.9176470588240.2549019607840.517647058824
1,Airventilation_CrossSection,234,65,132,255,0.917647,0.254902,0.517647,0.9176470588240.2549019607840.517647058824
2,AirventilationGrid_A,234,65,132,255,0.917647,0.254902,0.517647,0.9176470588240.2549019607840.517647058824
3,AirventilationGrid_B,234,65,132,255,0.917647,0.254902,0.517647,0.9176470588240.2549019607840.517647058824
4,Books,63,69,138,255,0.247059,0.270588,0.541176,0.2470588235290.2705882352940.541176470588
5,Bookshelf,83,102,202,255,0.325490,0.400000,0.792157,0.3254901960780.40.792156862745
6,Bottle,216,236,152,255,0.847059,0.925490,0.596078,0.8470588235290.9254901960780.596078431373
7,Box_Black,85,243,186,255,0.333333,0.952941,0.729412,0.3333333333330.9529411764710.729411764706
8,Box_Cardboard,85,243,186,255,0.333333,0.952941,0.729412,0.3333333333330.9529411764710.729411764706
9,Box_Gray,85,243,186,255,0.333333,0.952941,0.729412,0.3333333333330.9529411764710.729411764706


In [84]:
# Compare labels between correspondences:
total_correspondences = len(correspondences)
# Initialize dictionaries to 0:
total_positive_matches = {i:0 for i in hashed_df['hash']}
total_negative_matches = copy.deepcopy(total_positive_matches)

In [85]:
for correspondence in correspondences:
    est_pcl_color = est_pcl.colors[correspondence[0]]
    color_hash = str(est_pcl_color[0])+str(est_pcl_color[1])+str(est_pcl_color[2])
    if np.array_equal(est_pcl_color, gt_pcl.colors[correspondence[1]]):
        total_positive_matches[color_hash] += 1
    else:
        total_negative_matches[color_hash] += 1
        
print total_negative_matches
print total_positive_matches
print total_correspondences
assert(total_correspondences == total_negative_matches + total_positive_matches)

KeyError: '0.007843137254901960.4039215686274510.2784313725490196'